In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**ANN**

In [2]:
# -*- coding: utf-8 -*-
"""ANN.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1mJQdCg7-44XSmubsyjzdU_CmUk2AyBo_
"""

# Commented out IPython magic to ensure Python compatibility.
#import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Function to preprocess the dataset to array
def toArray(data_set):
  x_train = data_set.iloc[:108,[1,4,7,16,22,23,28,33]].copy()
  x_test = data_set.iloc[108:,[1,4,7,16,22,23,28,33]].copy()
  y_train = data_set.loc[:107,'No of Tourist(I'].copy() + data_set.loc[:107,'Foreign Tourist'].copy()
  y_test = data_set.loc[107:,'No of Tourist(I'].copy() + data_set.loc[107:,'Foreign Tourist'].copy()
  x_train = x_train.to_numpy()
  x_test = x_test.to_numpy()
  y_train = y_train.to_numpy()
  y_test = y_test.to_numpy()
  return x_train,x_test,y_train,y_test

#Read the csv file
df = pd.read_csv('/content/open-IIT-DATA - Sheet1 2.csv')
df

#Define the model
model = Sequential(
    [
        Dense(units=8, activation = 'relu'),
        Dense(units=4, activation = 'relu'),
        Dense(units=1, activation = 'linear')

    ], name = "my_model"
)

model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

X_train,X_test,y_train,y_test = toArray(df)

X_train.shape

y_train.shape

X_train

y_train

model.fit(
    X_train,y_train,
    epochs = 500
)

predictions = model.predict(X_test)

#Define the metric system for evaluation
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
def me(predictions, targets):
    return (abs(predictions-targets)).mean()

rmse(predictions, y_test)

me(predictions,y_test)

plt.plot(predictions, color='b')
plt.plot(y_test,color='r')

#Plotting the weights vs features to get the weightage of each feature in predicting the target data
x= model.weights

x = np.array(x)

x.shape

plt.plot(x[0])

plt.plot(x[2])

plt.plot(x[1])

predictions = np.squeeze(predictions)

predictions.shape

df['predicted'] = np.zeros(132)

for i in range(132):
  if i<109:
    df.loc[i,['predicted']] = None
  else:
    df.loc[i,['predicted']] = predictions[i-109]

df

df.set_index('Timestamp', inplace = True)

import matplotlib.ticker as plticker

loc = plticker.MultipleLocator(base=30.0) # this locator puts ticks at regular intervals

#Final plot
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
fig, ax = plt.subplots()
plt.plot(df['predicted'],color='orange', label='Predicted np. of tourists')
plt.plot(df['No of Tourist(I'], label='Actual no. of tourists')
ax.xaxis.set_major_locator(loc)
plt.xlabel('Time')
plt.ylabel('No. of Tourists')
plt.legend()
plt.show()





FileNotFoundError: [Errno 2] No such file or directory: '/content/open-IIT-DATA - Sheet1 2.csv'

**KNN**

In [ ]:
# -*- coding: utf-8 -*-
"""KNN code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1cx02N62OwZq-Ja8RdV7-vGIobXYRzjVS
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Data/Lagged.csv')
df1 = pd.read_csv('/content/drive/MyDrive/Data/data_tourist.csv')

try:
  df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y')
except ValueError:
  print("Error: Date format mismatch in your dataset.")

df['Numerical_Date'] = df.index + 1
df1['Numerical_Date'] = df1.index + 1
x=df['Numerical_Date'].values
y=df['SUM'].values
x1= df1['Numerical_Date'].values
y1 = df1['No of Tourist'].values
df.head()

#Implementing KNN
import numpy as np

# Define a function to calculate the Euclidean distance between two points.
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

class KNNRegressor:
    def __init__(self, k=3):
        self.k = k

    # Method to fit the KNN model with training data.
    def fit(self, X, y):
        # Store the training data and labels.
        self.X_train = X
        self.y_train = y

    # Method to make predictions for a set of input data points.
    def predict(self, X):
        # For each data point in X, make a prediction and return an array of predicted values.
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    # Internal method to predict a single data point 'x'.
    def _predict(self, x):
        # Calculate the Euclidean distance between the input point 'x' and all training data points.
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]

        # Get the indices of the 'k' nearest data points from the training data.
        k_indices = np.argsort(distances)[:self.k]

        # Get the labels (target values) of the 'k' nearest neighbors.
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # Return the mean (average) of the labels of the 'k' nearest neighbors as the predicted value for 'x'.
        return np.mean(k_nearest_labels)

# Split the dataset into training and testing sets for the first set of features and labels.
# The 'random_state' parameter ensures reproducibility of the split.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.2, random_state=42)

# Adjust the scale of 'y_test' by dividing it by 10,000. This may be a data-specific transformation.
y_test = y_test / 10000

# Create an instance of the KNNRegressor class with 'k' set to 3 (number of neighbors to consider).
knn_regressor = KNNRegressor(k=3)

# Fit the KNN model with the training data (X_train and y_train).
knn_regressor.fit(X_train, y_train)

# Adjust the scale of 'y_pred' by dividing it by 10,000.
y_pred = knn_regressor.predict(X_test)
y_pred = y_pred / 10000

# Calculate the Mean Absolute Error (MAE) as the mean absolute difference between 'y_test' and 'y_pred'.
mae = np.mean(np.abs(y_test - y_pred))

# Calculate the Root Mean Square Error (RMSE) as the square root of the mean squared difference between 'y_test' and 'y_pred'.
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Square Error (RMSE): {rmse}')

predicted_values = y_pred
y_test1 = y_test1/10000
Actual = y_test1

#Plotting the results
plt.figure(figsize= (10, 6))
plt.title(" Tourist and search index with 1 lag")
plt.plot( xt , Actual , label= 'No. of tourist'  )

plt.plot(xt , predicted_values , label= 'Search index + tourist ' )

plt.xticks(rotation=90)

plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.show()

**SARIMA**

In [ ]:
# -*- coding: utf-8 -*-
"""TouristPrediction.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1olUmGxJaBwHrlKBu-6PCwAjAlmSR793a
"""

# Commented out IPython magic to ensure Python compatibility.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline

from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/content/open-IIT-DATA - Sheet1 (1).csv', index_col='Timestamp')

scaler=StandardScaler()
scaler.fit(df[['weather shimla: (Worldwide)','Adventure activities: (Worldwide)','Shimla to manali: (Worldwide)','Shimla Airport']])

scaled_data=scaler.transform(df[['weather shimla: (Worldwide)','Adventure activities: (Worldwide)','Shimla to manali: (Worldwide)','Shimla Airport']])

pca=PCA(n_components=1)

pca.fit(scaled_data)

x_pca=pca.transform(scaled_data)

df['pca']= x_pca

def calc_rmse_mae(actual_data,predicted_data):
  mae = mean_absolute_error(actual_data, predicted_data)

  rmse = np.sqrt(mean_squared_error(actual_data, predicted_data))

  print(f"MAE: {mae}")
  print(f"RMSE: {rmse}")

df['No of Tourist'] = df['No of Tourist']/10000
df['Foreign Tourist'] = df['Foreign Tourist']/1000

df['pca_sum']=df['Hotels in shimla']+df['Cheap hotels ']+df['budget hotels ']+df['Travel guide']+df['Shimla flights: (Worldwide)']+df['train shimla: (Worldwide)']+df['Shopping Shimla']+df['Shimla maps: (Worldwide)']+df['pca']+df['No of Tourist']

df['Sum'] = df['Sum']/12.5
df['pca_sum'] = df['pca_sum']/12.5

df['Hotels in shimla Sum'] = (df['No of Tourist']+df['Hotels in shimla'])/2
df['Cheap hotels Sum'] = (df['No of Tourist']+df['Cheap hotels '])/2.5
df['budget hotels Sum'] = (df['No of Tourist']+df['budget hotels '])/1.4
df['Travel guide Sum'] = (df['No of Tourist']+df['Travel guide'])/2
df['Shimla flights: (Worldwide) Sum'] = (df['No of Tourist']+df['Shimla flights: (Worldwide)'])/3
df['train shimla: (Worldwide) Sum'] = (df['No of Tourist']+df['train shimla: (Worldwide)'])/3
df['Shopping Shimla Sum'] = (df['No of Tourist']+df['Shopping Shimla'])/3
df['Shimla maps: (Worldwide) Sum'] = (df['No of Tourist']+df['Shimla maps: (Worldwide)'])/2

df_train = df[0:100].copy()
df_test = df[100:132].copy()

adf, pval, usedlag, nobs, crit_vals, icbest =  adfuller(df_train['No of Tourist'])
print('ADF test statistic:', adf)
print('ADF p-values:', pval)
print('ADF number of lags used:', usedlag)
print('ADF number of observations:', nobs)
print('ADF critical values:', crit_vals)
print('ADF best information criterion:', icbest)

df_train['Seasonal Difference']=df_train['No of Tourist']-df_train['No of Tourist'].shift(8)

adf, pval, usedlag, nobs, crit_vals, icbest =  adfuller(df_train['Seasonal Difference'].dropna())
print('ADF test statistic:', adf)
print('ADF p-values:', pval)
print('ADF number of lags used:', usedlag)
print('ADF number of observations:', nobs)
print('ADF critical values:', crit_vals)
print('ADF best information criterion:', icbest)

df_train['Seasonal Difference'].plot(figsize=(12,8))

plot_pacf(df_train['Seasonal Difference'].dropna());
plot_acf(df_train['Seasonal Difference'].dropna());

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['No of Tourist'].dropna(),order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('number of tourists ')
plt.ylabel('Normalized number of tourists')
plt.legend()

calc_rmse_mae(df['No of Tourist'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['Hotels in shimla Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('Hotels in shimla + No of tourists')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['Hotels in shimla Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['budget hotels Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('budget hotels + No of tourists')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['budget hotels Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['Cheap hotels Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('cheap hotels + No of tourists')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['Cheap hotels Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['Travel guide Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('Travel guide + No of tourists')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['Travel guide Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['Shimla flights: (Worldwide) Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('Shimla flights: (Worldwide) + No of tourists')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['Shimla flights: (Worldwide) Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['train shimla: (Worldwide) Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('train shimla: (Worldwide) + No of tourists')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['train shimla: (Worldwide) Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['Shopping Shimla Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('Shopping Shimla + No of tourists')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['Shopping Shimla Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['Shimla maps: (Worldwide) Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['No of Tourist'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('Shimla maps: (Worldwide) + No of tourists')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['Shimla maps: (Worldwide) Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['Sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['Sum'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('Search Index')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['Sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

model_SARIMA=sm.tsa.statespace.SARIMAX(df_train['pca_sum'],order=(2, 1, 3),seasonal_order=(2,1,3,12))
results_SARIMA=model_SARIMA.fit()
plt.figure(figsize=(12,8))
df['pca_sum'].plot()
plt.plot(results_SARIMA.predict(start=100,end=131,dynamic=True) , label='predicted')
plt.title('Search Index with PCA')
plt.ylabel('Normalized volume')
plt.legend()

calc_rmse_mae(df['pca_sum'][100:132],results_SARIMA.predict(start=100,end=131,dynamic=True))

**SARIMAX**

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# #                                           Sarimax Model

# ####  Importing the required libraries 

# In[1]:


from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime
from datetime import timedelta
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf , plot_pacf
from time import time
import warnings
warnings.filterwarnings('ignore')
RANDOM_SEED = np.random.seed(0)


# ####   Importing the  dataset  as shimla

# In[2]:


# dataset below is the data of summation of all the best features including their lagged value 


# In[36]:


shimla = pd.read_csv("D:\DATA ANALYYSIS\data analysis\.ipynb_checkpoints\datasets\open_new_iit.csv", index_col = 'Timestamp')
dataset = shimla.iloc[:,[29]]
dataset= dataset/10
shimla.head()


# In[4]:


# dataset2 is data of Number of Tourist 


# In[42]:


dataset2 = shimla.iloc[:,[26]]
dataset2 = dataset2/1000
print(dataset2)


# #### splitting the dataset into test and training set (20:80)

# In[23]:


train = dataset.iloc[:105,:]
test = dataset.iloc[106:,:]
train_exp=dataset.iloc[:50,:]

plt.figure(figsize=(12,8))
plt.plot(train_exp.dropna(),label ='actual')
plt.title('HOTELS IN SHIMLA ',fontsize=20)
plt.ylabel('SEARCHES',fontsize=16)


# ####  using the seasonal decompose module from stats model library to find seasonilty trend and residue 

# In[21]:


plt.rc('figure',figsize=(12,8))
plt.rc('font',size=15)

result = seasonal_decompose(train_exp['sum features.1'].dropna(),model='additive', period= 7)
fig = result.plot()


# ####  plotting acf and pacf plots to find parameters of sarimax

# In[8]:


plot_acf(train.dropna())
plot_pacf(train.dropna())


# ####  training the sarimax model 

# In[9]:


import statsmodels.api as sm
model= sm.tsa.statespace.SARIMAX(train.dropna(),order=(0,0,0),seasonal_order=(1,1,1,12))
results=model.fit()


# In[49]:


plt.figure(figsize=(12,8))
import matplotlib.ticker as plticker
loc = plticker.MultipleLocator(base=30)
fig,ax=plt.subplots()
ax.xaxis.set_major_locator(loc)
plt.plot(dataset,label='Search Index + Tourist')
plt.plot(results.predict(start=104,end=131,dynamic=True) , label='Predicted')
plt.title('Tourist and Search Index with 0 and 1 lag ')
plt.ylabel('Normalised Volume')
plt.legend(loc='upper right')


# #### rmse and mae to evaluate the model 

# In[46]:


from sklearn.metrics import mean_squared_error
from math import sqrt
 
sqrt(mean_squared_error(test.dropna(), results.predict(start=106,end=131,dynamic=True))) 


# In[47]:


from sklearn.metrics import mean_absolute_error
mean_absolute_error(test.dropna(), results.predict(start=106,end=131,dynamic=True))


# In[48]:


np.mean(dataset)


# In[ ]:






**EXPONENTIAL SMOOTHING**

In [ ]:
# -*- coding: utf-8 -*-
"""Arima_Model1.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1aiX0gtBqCrx0iRoceTz3yw5X7HNy1tNP
"""



import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# Load the first time series data
df = pd.read_csv('/content/data_tourist.csv', index_col='Timestamp')
df = df.dropna()
print("Shape of Data", df.shape)

# Plot the original time series data
df.plot(figsize=(12, 5))

# Perform Augmented Dickey-Fuller test for stationarity
from statsmodels.tsa.stattools import adfuller
adf, pval, usedlag, nobs, crit_vals, icbest = adfuller(df.No_of_Tourist.values)
print('ADF test statistic:', adf)
print('ADF p-values:', pval)
print('ADF number of lags used:', usedlag)
print('ADF number of observations:', nobs)
print('ADF critical values:', crit_vals)
print('ADF best information criterion:', icbest)

# Differencing to make the time series stationary
prev_No_of_Tourist = df.No_of_Tourist.shift()
df = df.No_of_Tourist - prev_No_of_Tourist
df.plot(figsize=(12, 5))

# Check for missing values
df.isna()

# Prepare the training and test data
train = df['No_of_Tourist'].iloc[:-30] / 10000.0
test = df['No_of_Tourist'].iloc[-30:] / 10000.0

# Create an Exponential Smoothing model
model = sm.tsa.ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
model = model.fit()
model.summary()

# Make predictions
start = len(train)
end = len(train) + len(test) - 1
pred = model.predict(start=start, end=end).rename('Predictions')
pred.index = df.index[start:end + 1]

# Plot the test data and predictions
test.plot(legend=True, figsize=(8, 5))
pred.plot(legend=True)

# Calculate MAE and RMSE
rmse = np.sqrt(mean_squared_error(test, pred))
print("MAE:", mean_absolute_error(test, pred))
print("RMSE:", rmse)

# Load the second time series data
df = pd.read_csv('/content/open_new_iit.csv', index_col='Timestamp')
df = df.dropna()
print("Shape of Data", df.shape)

# Prepare the training and test data
train = df['sum(lag)'].iloc[:-30] / 10.0
test = df['sum(lag)'].iloc[-30:] / 10.0

# Create an Exponential Smoothing model
model = sm.tsa.ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
model = model fit()
model.summary()

# Make predictions
start = len(train)
end = len(train) + len(test) - 1
pred = model.predict(start=start, end=end).rename('Predictions')
pred.index = df.index[start:end + 1]

# Plot the test data and predictions
ax = test.plot(legend=True, figsize=(8, 5), label='Search Trend (with lag)')
ax.set_ylabel("Normalized Volume")
pred.plot(legend=True)

# Calculate MAE and RMSE
rmse = np.sqrt(mean_squared_error(test, pred))
print("MAE:", mean_absolute_error(test, pred))
print("RMSE:", rmse)

# Load the third time series data
train = df['Sum'].iloc[:-30] / 10.0
test = df['Sum'].iloc[-30:] / 10.0

# Create an Exponential Smoothing model
model = sm.tsa.ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
model = model.fit()
model.summary()

# Make predictions
start = len(train)
end = len(train) + len(test) - 1
pred = model.predict(start=start, end=end).rename('Predictions')
pred.index = df.index[start:end + 1]

# Plot the test data and predictions
ax = test.plot(legend=True, figsize=(8, 5), label='Search Trend')
ax.set_ylabel("Normalized Volume")
pred.plot(legend=True)

# Calculate MAE and RMSE
rmse = np.sqrt(mean_squared_error(test, pred))
print("MAE:", mean_absolute_error(test, pred))
print("RMSE:", rmse)



**CATBOOST**

In [ ]:
import catboost
import pandas as pd
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
df=pd.read_csv("D:\openiitda\SUMtotal.csv") #extract the total sum
ht=pd.read_csv("D:\openiitda\data_tourist 1.csv") #extract number of tourist
try:
    df['Timestamp']=pd.to_datetime(df['Timestamp'],format='%d-%m-%Y') #convert timestamp to numerical date
except ValueError:
    print("Error")
df['Numerical_date']=df['Timestamp'].dt.month
try:
    ht['Timestamp']=pd.to_datetime(ht['Timestamp'],format='%d/%m/%Y')
except ValueError:
    print("Error")
ht['Numerical_date']=ht['Timestamp'].dt.month 
X=df.drop('SUM',axis=1)	 #segregate x and y
y=df['SUM']
Xt=ht.drop('No of Tourist(I',axis=1)	
yt=ht['No of Tourist(I']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #test train split
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='RMSE')
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=100, verbose=100) #train the model
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, yt, test_size=0.2, random_state=42)
y_pred = model.predict(X_test) #predict using train data
y_pred
yt=yt_test
mse = mean_squared_error(y_test, y_pred) #calculate mean squared error
print(f'Mean Squared Error (MSE): {mse}')
from math import sqrt
print(sqrt(mse))
from sklearn.metrics import mean_absolute_error #calculate mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
import matplotlib.pyplot as plt

actual_values=yt    #plot graph between tourist and SUM
predicted_values=y_pred


time_index=range(len(actual_values))

plt.figure(figsize=(10,6))
plt.title("Tourists and search index with lag")
plt.plot(time_index, actual_values, label='Number of Tourists',  marker='')
plt.plot(time_index, predicted_values, label='Tourists+search trends(with lag)', color='orange', marker='')

plt.xlabel("Time")
plt.ylabel("Normalised volumes")
plt.legend()
plt.show()


**LSTM**

In [ ]:
# -*- coding: utf-8 -*-
"""RNN_LSTM.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wShfrNd72saKh86F22nF7d4Xihr17Rx4
"""

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

# Load data from a CSV file
data = pd.read_csv('/Users/vedantahazra/Downloads/open-IIT-DATA - Sheet1 2.csv')

# Select relevant columns
cols = ['Timestamp','No of Tourist(I']
data = data[cols]
data

# Check if a GPU is available, and set the device accordingly
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

from copy import deepcopy as dc

# Define a function to prepare the dataframe for LSTM
def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)

    df.set_index('Timestamp', inplace=True)

    # Create lag features for time series data
    for i in range(1, n_steps+1):
        df[f'No of Tourist(I((t-{i})'] = df['No of Tourist(I'].shift(i)

    # Remove rows with NaN values
    df.dropna(inplace=True)

    return df

# Set the number of lookback steps for time series
lookback = 7

# Prepare the dataframe for LSTM
shifted_df = prepare_dataframe_for_lstm(data, lookback)
shifted_df

# Convert the dataframe to a NumPy array
shifted_df_as_np = shifted_df.to_numpy()

shifted_df_as_np

shifted_df_as_np.shape

# Apply Min-Max scaling to the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

shifted_df_as_np = scaler.fit_transform(shifted_df_as_np)

shifted_df_as_np

# Separate the features (X) and target (y)
X = shifted_df_as_np[:, 1:]
y = shifted_df_as_np[:, 0]

X.shape, y.shape

# Flip the features along the time axis
X = dc(np.flip(X, axis=1))
X

# Split the data into training and testing sets
split_index = int(len(X) * 0.8)

split_index

X_train = X[:split_index]
X_test = X[split_index:]

y_train = y[:split_index]
y_test = y[split_index:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Reshape the data for compatibility with the LSTM model
X_train = X_train.reshape((-1, lookback, 1))
X_test = X_test.reshape((-1, lookback, 1))

y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

from torch.utils.data import Dataset

# Define a custom dataset for PyTorch
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

train_dataset

from torch.utils.data import DataLoader

# Create DataLoader objects for training and testing
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

# Define an LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        # Create an LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers,
                            batch_first=True)

        # Create a fully connected (linear) layer for prediction
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Initialize the LSTM model
model = LSTM(1, 6, 1)
model.to(device)
model

# Define training and validation functions
def train_one_epoch():
    # Set the model in training mode
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

# Set the model in evaluation mode
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

# Set hyperparameters for training
learning_rate = 0.001
num_epochs = 500
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

# Make predictions on the training and testing datasets
with torch.no_grad():
    predicted = model(X_train.to(device)).to('cpu').numpy()

# Visualize the results
plt.plot(y_train, label='Actual Tourists')
plt.plot(predicted, label='Predicted Tourists')
plt.xlabel('Month')
plt.ylabel('Tourists')
plt.legend()
plt.show()

train_predictions = predicted.flatten()

dummies = np.zeros((X_train.shape[0], lookback+1))
dummies[:, 0] = train_predictions
dummies = scaler.inverse_transform(dummies)

train_predictions = dc(dummies[:, 0])
train_predictions

# Inverse transform the actual values to the original scale
dummies = np.zeros((X_train.shape[0], lookback+1))
dummies[:, 0] = y_train.flatten()
dummies = scaler.inverse_transform(dummies)

new_y_train = dc(dummies[:, 0])
new_y_train

plt.plot(new_y_train, label='Actual Tourists')
plt.plot(train_predictions, label='Predicted Tourists')
plt.xlabel('Month')
plt.ylabel('Tourists')
plt.legend()
plt.show()

test_predictions = model(X_test.to(device)).detach().cpu().numpy().flatten()

dummies = np.zeros((X_test.shape[0], lookback+1))
dummies[:, 0] = test_predictions
dummies = scaler.inverse_transform(dummies)

test_predictions = dc(dummies[:, 0])
test_predictions

dummies = np.zeros((X_test.shape[0], lookback+1))
dummies[:, 0] = y_test.flatten()
dummies = scaler.inverse_transform(dummies)

new_y_test = dc(dummies[:, 0])
new_y_test

plt.plot(new_y_test, label='Actual Tourists')
plt.plot(test_predictions, label='Predicted Tourists')
plt.xlabel('Month')
plt.ylabel('Tourists')
plt.legend()
plt.show()

#Define the metric system for evaluation

#Root mean squared error
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

#Mean absolute error
def me(predictions, targets):
    return (abs(predictions-targets)).mean()

#rmse of test predictions
rmse(test_predictions,new_y_test)

#me of test predictions
me(test_predictions,new_y_test)

#rmse of train data
rmse(new_y_train,train_predictions)

#me of train data
me(new_y_train,train_predictions)

#checking the model parameters
model.parameters

m = train_predictions.shape[0] + test_predictions.shape[0]

predictions = np.zeros(m)

for i in range(m):
    if(i<train_predictions.shape[0]):
        predictions[i] = train_predictions[i]
    else:
        predictions[i] = test_predictions[i-train_predictions.shape[0]]

shifted_df['predictions'] = np.zeros(125)

shifted_df['Timestamp'] = shifted_df.index

index = np.zeros(125)
for i in range(125):
    index[i] = i
shifted_df['index'] = index
shifted_df.set_index('index', inplace = True)

for i in range(125):
    if i<100:
        shifted_df.loc[i,['predictions']] = None
    else:
        shifted_df.loc[i,['predictions']] = test_predictions[i-100]

shifted_df.set_index('Timestamp', inplace = True)

import matplotlib.ticker as plticker

loc = plticker.MultipleLocator(base=30.0) # this locator puts ticks at regular intervals

#Final plot
font = {'family':'serif','color':'black','size':12}
fig,ax = plt.subplots()
ax.xaxis.set_major_locator(loc)
plt.plot(shifted_df['No of Tourist(I'], label='Actual Tourists')
plt.plot(shifted_df['predictions'],color='orange', label='Predicted Tourists')
plt.xlabel('Time', fontdict = font)
plt.ylabel('Tourists', fontdict = font)
plt.legend()
plt.show()

shifted_df



**XGBOOST**

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
df = pd.read_csv("D:\openiitda\open_new_iit.csv") #open the file containing the data
try:
    df['Timestamp']=pd.to_datetime(df['Timestamp'],format='%d-%m-%Y') #convert timestamp to numerical date
except ValueError:
    print("Error")
df['Numerical_date']=df['Timestamp'].dt.month
X = df['Numerical_date'] 
y = df['SUM']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# test train split
dtrain = xgb.DMatrix(X_train, label=y_train) #fit the train test data into XGB matrix
dtest = xgb.DMatrix(X_test, label=y_test)
params = { #set the XGBoost parameters
    'objective': 'reg:squarederror',  # for regression
    'max_depth': 5,
    'learning_rate': 0.1,
    'n_estimators': 500,
}
model = xgb.train(params, dtrain, num_boost_round=10)  #train the model
y_pred = model.predict(dtest) #predict from the trained data
from sklearn.metrics import accuracy_score, mean_squared_error
mse = mean_squared_error(y_test, y_pred)  #calculate mean squared error
print(f'Mean Squared Error: {mse}')
from math import sqrt
rmse=sqrt(mse)
rmse
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)   #calculate mean absolute error
print(f"Mean Absolute Error: {mae}")
xt=df['Numerical_date']
yt=df['No of Tourist(I']
Xt_train, Xt_test, yt_train, yt_test = train_test_split(X, y, test_size=0.2, random_state=42) #test train split of number of tourists
import matplotlib.pyplot as plt

actual_values=yt_test
predicted_values=y_pred  #plot the graph

time_index=range(len(actual_values))

plt.figure(figsize=(12,8))
plt.title("Actual tourists vs predicted sum of tourists and search volumes")
plt.plot(time_index, actual_values, label='Number of tourists', color='blue', marker='')
plt.plot(time_index, predicted_values, label='Sum of trends', color='red', marker='')

plt.xlabel("Dates")
plt.ylabel("Normalised Volumes")
plt.legend()
plt.show()
